# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Potential Important Variables:
- USE(1-4)_DESC : property use types
- EMV_LAND, EMV_BLDG, EMV_TOTAL : est market value of land, buildings, total
- ACRES_POLY, ACRES_DEED : acreage of the land
- BASEMENT : basement (Y/N)
- GARAGE, GARAGESQFT : large concrete surface? garage is Y/N
- FIN_SQ_FT: sq footage
- YEAR_BUILT
- NUM_UNITS
- GREEN_ACRE, OPEN_SPACE, AG_PRESERV : undeveloped land / open areas
- TOTAL_TAX, TAX_CAPAC
- SPEC_ASSES : special assessments </font>

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> **Feature Construction Strategy:**

**Numerical Variables - Use mean, median, std, min, max:**
- `EMV_LAND`, `EMV_BLDG`, `EMV_TOTAL`: Market value summaries (mean, median, std) - higher values may indicate more development
- `ACRES_POLY`, `ACRES_DEED`: Property size summaries - larger lots may have more runoff area
- `GARAGESQFT`: Average garage size - indicates impervious surface area
- `FIN_SQ_FT`: Average building footprint - more impervious surface
- `YEAR_BUILT`: Mean/median year built - track development timeline
- `NUM_UNITS`: Total units, mean units per property - population density indicator
- `TOTAL_TAX`, `TAX_CAPAC`: Tax summaries - economic indicators
- `SPEC_ASSES`: Count/sum of special assessments - infrastructure improvements

**Binary/Categorical Variables - Use proportions/counts:**
- `USE1_DESC` (and USE2-4): Create indicator columns for common use types (residential, commercial, agricultural, industrial), then compute counts or proportions
- `BASEMENT`: Proportion with basements (Y vs N)
- `GARAGE`: Proportion with garages (Y vs N)
- `GREEN_ACRE`, `OPEN_SPACE`, `AG_PRESERV`: Count properties with these designations - less developed = potentially better water quality

**Aggregation Strategy:**
- Group by: `Monit_MAP_CODE1` (lake ID), `Year`, `distance_category` (within_500m, between_501_1600m, over_1600m)
- For each group, calculate the features above
- This gives us development characteristics at different distances from each lake over time
</font>